In [9]:
import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import pandas as pd
#import math as m
#from typing import List, Tuple, Optional
import matplotlib.pyplot as plt
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import mean_squared_error
from data_prep import Dataset,create_datasets, encode_time
from model import LSTM_multivariate_input_multi_step_forecaster
from lstm_functions import train_and_val


ImportError: cannot import name 'run_train_and_val' from 'lstm_functions' (c:\Users\storr\OneDrive - Danmarks Tekniske Universitet\Year 2\Semester 1\Deep Learning\Project\deeplearning\model_functions\lstm_functions.py)

In [ ]:
data_da_23 = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/deeplearning/models_santi/data_santi/data_da_23.csv')
data_id_23 = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/deeplearning/models_santi/data_santi/data_id_23.csv')
data_da_24 = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/deeplearning/models_santi/data_santi/data_da_24.csv')
data_id_24 = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/deeplearning/models_santi/data_santi/data_id_24.csv')
load_23_24 = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/deeplearning/models_santi/data_santi/load_data_2023_2024.csv',sep=';')

data_da = pd.concat([data_da_23,data_da_24],ignore_index=True)
data_id = pd.concat([data_id_23,data_id_24],ignore_index=True)

# retriving only prices and load from data (not date columns)
da_prices = data_da['DA ES']
id_prices = data_id.iloc[:,-3:]
load_23_24 = load_23_24['value']

# saving date features
date_features = pd.DataFrame()
for feature,max_value in zip(['Month','Day','Hour'],[12,31,24]):
  data_sin_feature = data_da[feature].apply(lambda x: m.sin(x*2*m.pi/max_value))
  data_cos_feature = data_da[feature].apply(lambda x: m.cos(x*2*m.pi/max_value))
  date_features = pd.concat([date_features,data_sin_feature,data_cos_feature],axis=1)

In [ ]:
# setting datasets
lag_in_days = 7
forecast_horizon_in_hours = 24

input_data = np.hstack([np.array(date_features),np.array(load_23_24).reshape(len(load_23_24),1),np.array(da_prices).reshape(len(da_prices),1),np.array(id_prices)])
output_data_DA = np.array(da_prices).reshape(len(da_prices),1)
output_data_ID = np.array(id_prices.iloc[:,-1]).reshape(len(id_prices),1)
output_DA_diff_ID = output_data_DA - output_data_ID

# Set transforms if wanted here

# Train,val and test sets
training_set_DA, val_set_DA, test_set_DA = create_datasets(input_data,output_data_DA,lag_in_days,forecast_horizon_in_hours,Dataset)
training_set_ID, val_set_ID, test_set_ID = create_datasets(input_data,output_data_ID,lag_in_days,forecast_horizon_in_hours,Dataset)
training_set_DA_diff_ID, val_set_DA_diff_ID, test_set_DA_diff_ID = create_datasets(input_data,output_DA_diff_ID,lag_in_days,forecast_horizon_in_hours,Dataset)

In [ ]:
# Creating models

# model inputs
input_size = training_set_DA.inputs.shape[2]
future_inputs_size = training_set_DA.future_inputs.shape[2]
past_horizon = training_set_DA.inputs.shape[1]
hidden_size = 50
num_layers = 2
forecast_horizon = training_set_DA.targets.shape[1]
dropout = 0.2

model_DA = LSTM_multivariate_input_multi_step_forecaster(input_size, hidden_size,num_layers,dropout,past_horizon,forecast_horizon,future_inputs_size)
model_ID = LSTM_multivariate_input_multi_step_forecaster(input_size, hidden_size,num_layers,dropout,past_horizon,forecast_horizon,future_inputs_size)
model_DA_diff_ID = LSTM_multivariate_input_multi_step_forecaster(input_size, hidden_size,num_layers,dropout,past_horizon,forecast_horizon,future_inputs_size)

optimizer_DA = optim.Adam(model_DA.parameters(),lr=0.001) # add momentum?
optimizer_ID = optim.Adam(model_ID.parameters(),lr=0.001)
optimizer_DA_diff_ID = optim.Adam(model_DA_diff_ID.parameters(),lr=0.001)

In [ ]:
# Running LSTM
num_epochs = 100
batch_size = 100
criterion = nn.MSELoss(reduction='mean')

# training DA model
train_loader_DA = torch.utils.data.DataLoader(
    training_set_DA,
    batch_size=batch_size,
    shuffle=False
)
val_loader_DA = torch.utils.data.DataLoader(
    val_set_DA,
    batch_size=1,
    shuffle=False
)

model_DA = train_and_val(train_loader_DA,val_loader_DA,num_epochs,model_DA,optimizer_DA,criterion)

In [ ]:
# Plotting results
forecasts_DA = []
targets_DA = []
model_DA.eval()
for inputs,target,future_inputs in val_loader_DA:
  inputs = inputs.float()
  target = target.float()
  future_inputs = future_inputs.float()
  forecast = model_DA(inputs,future_inputs)
  forecasts_DA.append(forecast.data)
  targets_DA.append(target.data)

